In [1]:
import pandas as pd 
import numpy as np 
import configparser
import os
import seaborn as sns
import plotly_express as px
import kaleido

import nltk 

config = configparser.ConfigParser()
config.read("../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
base_path = config['DEFAULT']['base_path']
code_dir = config['DEFAULT']['code_dir']

data_prefix = 'Maha'
out_path = f'{output_dir}/{data_prefix}'
OHCO = ['book_id','chap_id','sec_id','para_num', 'sent_num', 'token_num']


os.chdir(code_dir)
import preprocess
import bow_analysis
os.chdir(base_path + "\\notebooks")

In [2]:
CORPUS = pd.read_csv(rf"{output_dir}\F2\{data_prefix}-CORPUS.csv", sep="|").set_index(OHCO)
LIB = pd.read_csv(fr"{output_dir}\F2\{data_prefix}-LIB.csv", sep="|").set_index('book_id')
LIB2 = pd.read_csv(fr"{output_dir}\F2\{data_prefix}-LIB2.csv", sep="|").set_index(OHCO[:2])
VOCAB = pd.read_csv(rf"{output_dir}\F2\{data_prefix}-VOCAB.csv", sep="|").set_index("term_str")
VOCAB2 = pd.read_csv(rf"{output_dir}\CHAP_BOW\{data_prefix}-VOCAB2.csv", sep="|").set_index("term_str")
TFIDF = pd.read_csv(rf"{output_dir}\CHAP_BOW\{data_prefix}-TFIDF_REDUCED_CHAPS_L2.csv", sep="|").set_index(OHCO[:2])

In [3]:
TFIDF

shafts   pierced      army   deities  carwarriors  \
book_id chap_id                                                        
1       1        0.000000  0.017978  0.029045  0.000000     0.000000   
        2        0.000000  0.006462  0.037584  0.009168     0.000000   
        3        0.000000  0.000000  0.000000  0.013734     0.000000   
        4        0.000000  0.009420  0.000000  0.000000     0.000000   
        5        0.000000  0.020106  0.000000  0.032600     0.000000   
...                   ...       ...       ...       ...          ...   
15      2        0.000000  0.000000  0.000000  0.226051     0.013851   
        3        0.017530  0.000000  0.000000  0.000000     0.000000   
16      1        0.062153  0.014705  0.000000  0.041726     0.086291   
17      1        0.000000  0.000000  0.000000  0.201742     0.000000   
18      1        0.000000  0.000000  0.000000  0.296294     0.059203   

                   troops   sanjaya       cut  penances    partha  ...  \
book_id chap_id                                                    ...   
1       1        0.000000  0.674489  0.000000  0.000000  0.000000  ...   
        2        0.050112  0.049816  0.000000  0.000000  0.000000  ...   
        3        0.000000  0.000000  0.000000  0.000000  0.000000  ...   
        4        0.000000  0.000000  0.000000  0.022543  0.000000  ...   
        5        0.000000  0.000000  0.016690  0.099664  0.000000  ...   
...                   ...       ...       ...       ...       ...  ...   
15      2        0.030889  0.000000  0.000000  0.317740  0.000000  ...   
        3        0.000000  0.374090  0.000000  0.111660  0.000000  ...   
16      1        0.014255  0.000000  0.000000  0.035190  0.058821  ...   
17      1        0.000000  0.000000  0.000000  0.042535  0.017775  ...   
18      1        0.000000  0.000000  0.010552  0.043458  0.000000  ...   

                 propitiatory  apartment  anguish    thorn  cheered  \
book_id chap_id                                                       
1       1                 0.0   0.000000  0.01683  0.00000      0.0   
        2                 0.0   0.000000  0.00000  0.00000      0.0   
        3                 0.0   0.000000  0.00000  0.00000      0.0   
        4                 0.0   0.054643  0.00000  0.00000      0.0   
        5                 0.0   0.000000  0.00000  0.00833      0.0   
...                       ...        ...      ...      ...      ...   
15      2                 0.0   0.000000  0.00000  0.00000      0.0   
        3                 0.0   0.000000  0.00000  0.00000      0.0   
16      1                 0.0   0.000000  0.00000  0.00000      0.0   
17      1                 0.0   0.000000  0.00000  0.00000      0.0   
18      1                 0.0   0.000000  0.00000  0.00000      0.0   

                 believing     tops  pressing       hid    affair  
book_id chap_id                                                    
1       1              0.0  0.00000   0.00000  0.000000  0.000000  
        2              0.0  0.00000   0.00000  0.000000  0.000000  
        3              0.0  0.00000   0.00000  0.000000  0.000000  
        4              0.0  0.00000   0.00000  0.000000  0.027321  
        5              0.0  0.00833   0.00833  0.008066  0.000000  
...                    ...      ...       ...       ...       ...  
15      2              0.0  0.00000   0.00000  0.000000  0.000000  
        3              0.0  0.00000   0.00000  0.000000  0.000000  
16      1              0.0  0.00000   0.00000  0.000000  0.000000  
17      1              0.0  0.00000   0.00000  0.000000  0.000000  
18      1              0.0  0.00000   0.00000  0.000000  0.000000  

[99 rows x 3080 columns]

In [4]:
from sklearn.decomposition import PCA
n_comps = 10


pc_cols = [f"PC{i}" for i in range(n_comps)]
pca_engine = PCA(n_components=n_comps)
DCM = pd.DataFrame(pca_engine.fit_transform(TFIDF), index=TFIDF.index)
DCM.columns = ['PC{}'.format(i) for i in DCM.columns]
DCM

PC0       PC1       PC2       PC3       PC4       PC5  \
book_id chap_id                                                               
1       1        0.015683 -0.034806 -0.297601  0.177902  0.149307  0.544222   
        2       -0.105643  0.036544 -0.025023  0.008277  0.088218  0.181899   
        3       -0.173483  0.062936  0.080646 -0.077161 -0.068198  0.081600   
        4       -0.181120  0.192393  0.275758  0.247527  0.019174  0.069340   
        5       -0.150651 -0.042015  0.162823 -0.082515 -0.100916  0.029353   
...                   ...       ...       ...       ...       ...       ...   
15      2       -0.179403 -0.257225 -0.001430  0.065007  0.266729 -0.061380   
        3       -0.052475 -0.109117 -0.249248  0.212811  0.273728  0.192898   
16      1        0.008690  0.014393  0.012272 -0.155412  0.269955 -0.175635   
17      1       -0.147061 -0.082419  0.059136 -0.086736  0.210909  0.025072   
18      1       -0.192983 -0.312943  0.143894 -0.086099  0.139287  0.009691   

                      PC6       PC7       PC8       PC9  
book_id chap_id                                          
1       1        0.021703 -0.100023  0.099600  0.192281  
        2        0.009559 -0.053494 -0.040440 -0.010241  
        3       -0.056366 -0.022399 -0.079847 -0.109740  
        4        0.031206 -0.048664  0.047612  0.047855  
        5        0.004863 -0.151615 -0.092567  0.076375  
...                   ...       ...       ...       ...  
15      2       -0.267670  0.047423 -0.211391  0.025372  
        3       -0.313127 -0.061914 -0.153628  0.062978  
16      1       -0.320508 -0.052649  0.224145 -0.125594  
17      1       -0.118610  0.058711 -0.087638 -0.096378  
18      1       -0.068659  0.085026 -0.088060  0.055268  

[99 rows x 10 columns]

In [5]:
DCM = DCM.join(LIB2[['book_name','chap_name']], on=OHCO[:2])
DCM = DCM.join(LIB, on=OHCO[:1])
DCM

PC0       PC1       PC2       PC3       PC4       PC5  \
book_id chap_id                                                               
1       1        0.015683 -0.034806 -0.297601  0.177902  0.149307  0.544222   
        2       -0.105643  0.036544 -0.025023  0.008277  0.088218  0.181899   
        3       -0.173483  0.062936  0.080646 -0.077161 -0.068198  0.081600   
        4       -0.181120  0.192393  0.275758  0.247527  0.019174  0.069340   
        5       -0.150651 -0.042015  0.162823 -0.082515 -0.100916  0.029353   
...                   ...       ...       ...       ...       ...       ...   
15      2       -0.179403 -0.257225 -0.001430  0.065007  0.266729 -0.061380   
        3       -0.052475 -0.109117 -0.249248  0.212811  0.273728  0.192898   
16      1        0.008690  0.014393  0.012272 -0.155412  0.269955 -0.175635   
17      1       -0.147061 -0.082419  0.059136 -0.086736  0.210909  0.025072   
18      1       -0.192983 -0.312943  0.143894 -0.086099  0.139287  0.009691   

                      PC6       PC7       PC8       PC9  ... n_chaps n_secs  \
book_id chap_id                                          ...                  
1       1        0.021703 -0.100023  0.099600  0.192281  ...      19    235   
        2        0.009559 -0.053494 -0.040440 -0.010241  ...      19    235   
        3       -0.056366 -0.022399 -0.079847 -0.109740  ...      19    235   
        4        0.031206 -0.048664  0.047612  0.047855  ...      19    235   
        5        0.004863 -0.151615 -0.092567  0.076375  ...      19    235   
...                   ...       ...       ...       ...  ...     ...    ...   
15      2       -0.267670  0.047423 -0.211391  0.025372  ...       3     38   
        3       -0.313127 -0.061914 -0.153628  0.062978  ...       3     38   
16      1       -0.320508 -0.052649  0.224145 -0.125594  ...       1      7   
17      1       -0.118610  0.058711 -0.087638 -0.096378  ...       1      2   
18      1       -0.068659  0.085026 -0.088060  0.055268  ...       1      5   

                n_sents n_tokens  n_chars       parva_name  timeline  \
book_id chap_id                                                        
1       1         12422   228684  1055885              Adi   Pre-War   
        2         12422   228684  1055885              Adi   Pre-War   
        3         12422   228684  1055885              Adi   Pre-War   
        4         12422   228684  1055885              Adi   Pre-War   
        5         12422   228684  1055885              Adi   Pre-War   
...                 ...      ...      ...              ...       ...   
15      2          1742    30376   141565    Ashramavasika  Post-War   
        3          1742    30376   141565    Ashramavasika  Post-War   
16      1           432     8072    37596          Mausala  Post-War   
17      1           206     3062    14086  Mahaprasthanika  Post-War   
18      1           495     8628    40381     Svargarohana  Post-War   

                                                         key_focus  \
book_id chap_id                                                      
1       1            Origins, family lineages, childhood of heroes   
        2            Origins, family lineages, childhood of heroes   
        3            Origins, family lineages, childhood of heroes   
        4            Origins, family lineages, childhood of heroes   
        5            Origins, family lineages, childhood of heroes   
...                                                            ...   
15      2                           Retirement and death of elders   
        3                           Retirement and death of elders   
16      1              Destruction of Yadava clan, Krishna's death   
17      1           Pandavas renounce kingdom, begin final journey   
18      1        Yudhishthira’s ascent to heaven and cosmic res...   

                   main_theme   broad_theme  
book_id chap_id                              
1       1             line

In [6]:
DCM.to_csv(fr"{output_dir}\PCA_Results\{data_prefix}-PCA10_DCM.csv",sep = "|")

In [7]:
LOADINGS = pd.DataFrame(pca_engine.components_.T * np.sqrt(pca_engine.explained_variance_))
LOADINGS.index = TFIDF.columns
LOADINGS.columns = ["PC{}".format(i) for i in LOADINGS.columns]
LOADINGS.index.name = 'term_str'
LOADINGS2 = LOADINGS.join(VOCAB2, on="term_str")
LOADINGS2

,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,n_pos,cat_pos,df,idf,dp,di,dh,dfidf,tfidf_mean,tfidf_sum
term_str,,,,,,,,,,,,,,,,,,,,,
shafts,0.081497,-0.007962,0.016142,-0.004876,-0.007532,-0.007923,0.005197,-0.010465,-0.004612,-0.006080,...,11,"{'JJ', 'RB', 'VB', 'NNP', 'VBP', 'NN', 'VBD', ...",57.0,0.796467,0.575758,0.796467,0.458572,45.398597,0.000569,0.056298
pierced,0.050607,-0.006920,0.010463,-0.001360,-0.004356,-0.004414,0.005691,-0.005985,-0.002184,-0.004118,...,5,"{'JJ', 'NNP', 'VBD', 'NN', 'VBN'}",55.0,0.847997,0.555556,0.847997,0.471109,46.639830,0.000360,0.035682
army,0.038258,-0.007248,-0.005276,0.000791,-0.002680,0.002077,0.006410,0.002127,0.005454,0.005379,...,8,"{'JJ', 'NNP', 'VBP', 'IN', 'NN', 'VBD', 'VBZ',...",56.0,0.822002,0.565657,0.822002,0.464971,46.032095,0.000389,0.038510
deities,-0.019488,-0.043401,0.019990,-0.010421,0.014787,-0.007841,-0.004826,0.008892,-0.007523,0.006911,...,5,"{'JJ', 'NNP', 'VBZ', 'NNPS', 'NNS'}",43.0,1.203092,0.434343,1.203092,0.522555,51.732950,0.000281,0.027864
carwarriors,0.054079,-0.008123,0.001432,0.003623,0.003587,-0.007656,0.000494,0.002134,-0.000685,-0.003191,...,2,"{'NNP', 'NNS'}",46.0,1.105795,0.464646,1.105795,0.513804,50.866555,0.000441,0.043627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
believing,-0.000406,-0.000248,-0.000523,0.000403,-0.000895,-0.000203,0.000606,-0.000876,0.000174,-0.000011,...,2,"{'NN', 'VBG'}",18.0,2.459432,0.181818,2.459432,0.447169,44.269769,0.000023,0.002284
tops,-0.000454,-0.000058,-0.001459,0.000272,-0.000311,0.000629,-0.000115,-0.000759,-0.000608,0.000104,...,1,{'NNS'},18.0,2.459432,0.181818,2.459432,0.447169,44.269769,0.000036,0.003612
pressing,0.000152,0.003209,0.001030,0.003700,0.001796,-0.002065,0.002465,0.001366,-0.001284,0.000977,...,1,{'VBG'},18.0,2.459432,0.181818,2.459432,0.447169,44.269769,0.000055,0.005441


In [8]:
LOADINGS2.to_csv(fr"{output_dir}\PCA_Results\{data_prefix}-PCA10_LOADINGS.csv",sep = "|")

In [9]:
def vis_pcs(M, a, b,color ="book_name" ,hover_name='chap_name'):
    M["book_name"] = pd.Categorical(M["book_name"], ordered=True)
    fig =  px.scatter(M, f"PC{a}", f"PC{b}", 
                      color = color ,
                      title = f"PCA Components Visualization - PC{a+1} vs PC{b+1}",
                    hover_name=hover_name, 
                    # symbol=symbol, size=size,
                    marginal_x='box', height=800, width = 1008)
    
    fig.update_layout(
    xaxis_title=f'PC{a+1}',
    yaxis_title=f'PC{b+1}'
    )

    return fig

def vis_loadings(LOADINGS, a=0, b=1, hover_name='term_str'):
    fig= px.scatter(LOADINGS.reset_index(), f"PC{a}", f"PC{b}", 
                      title = f"PCA Loadings Visualization - PC{a+1} vs PC{b+1}",
                      text='term_str', hover_name='term_str',
                       color='max_pos_group', size="dfidf",
                      marginal_x='box', height=800)
    
    fig.update_layout(
    xaxis_title=f'PC{a+1}',
    yaxis_title=f'PC{b+1}'
    )

    return fig

In [22]:
fig01pcs = vis_pcs(DCM, 0, 1, "timeline")
fig01pcs.write_html(base_path + r"/plots/PC12.html")
# fig01pcs.write_image(fr"{output_dir}\PCA_Results\{data_prefix}-PCA10_01PCS.png")
fig01pcs.show()
base_path

'D:\\MSDS\\Spring Term\\DS5001 - Text Mining\\Mahabharata_NLP'

In [24]:
fig01load = vis_loadings(LOADINGS2, 0, 1)
fig01load.write_html(base_path + r"/plots/PC12_load.html")
fig01load.show()

In [23]:
fig12pcs = vis_pcs(DCM, 1, 2, "timeline")
fig12pcs.write_html("../plots/PC23.html")
fig12pcs.show()

In [25]:
fig12load = vis_loadings(LOADINGS2, 1, 2)
fig12load.write_html("../plots/PC23_load.html")
fig12load.show()

In [14]:
top_terms_sk= {}

data = []
for i in range(n_comps):
    for j in [0, 1]:
        data.append((f"PC{i}", j, ' '.join(LOADINGS.sort_values(f'PC{i}', ascending=bool(j)).head(10).index.to_list())))

comp_strs = pd.DataFrame(data)
comp_strs.columns =  ['pc', 'pole', 'top_terms']
comp_strs = comp_strs.set_index(['pc', 'pole'])


COMPS = comp_strs
COMPS['top_doc'] = pd.concat([DCM[pc_cols].idxmax(), DCM[pc_cols].idxmin()], axis=0, keys=[0,1])\
    .to_frame('top_doc')\
    .reset_index()\
    .rename(columns={'level_0':'pole', 'level_1':'pc'})\
    .set_index(['pc','pole'])\
    .sort_index()

In [15]:
COMPS

top_terms  top_doc
pc  pole                                                            
PC0 0     shafts carwarriors pierced sanjaya army cut dr...   (7, 6)
    1     deities vidura penances vyasa brahman becomes ...  (13, 1)
PC1 0     rakshasa jarasandha monarchs virata sisupala r...   (3, 2)
    1     deities righteousness becomes penances brahman...  (12, 3)
PC2 0     rakshasa deities shafts bhrigu rama vasishtha ...   (1, 9)
    1     sanjaya vidura gandhari dussasana hope dhritar...   (5, 2)
PC3 0     rakshasa sanjaya vidura gandhari bhrigu hope s...   (3, 2)
    1     jarasandha sisupala monarchs vrishni partha an...  (1, 18)
PC4 0     jarasandha rakshasa gandhari sanjaya magadha d...   (2, 3)
    1     virata dice dussasana play spake thereupon mat...   (3, 6)
PC5 0     sanjaya island hope islands chariot yaksha ser...   (1, 1)
    1     vidura rakshasa drupada town vrishnis monarchs...  (1, 14)
PC6 0     jarasandha magadha rakshasa sisupala monarchs ...   (2, 3)
    1     gandhari ladies andhakas vrishnis rama virata ...  (16, 1)
PC7 0     virata matsyas viratas jarasandha uttara matsy...   (4, 2)
    1     vidura sanjaya maya shafts mansion phalguna ba...   (1, 8)
PC8 0     rama andhakas kesava vrishnis sanjaya vrishni ...  (1, 18)
    1     jarasandha gandhari vidura household ladies pe...   (2, 4)
PC9 0     sanjaya rakshasa virata rama vasishtha maiden ...   (6, 1)
    1     savitri markandeya vidura misfortune attaineth...  (3, 14)

In [16]:
TFIDF.loc[(7, 6)].sort_values(ascending=False).head(20)

term_str
shafts         0.389722
satyaki        0.316299
pierced        0.219966
sindhus        0.217252
carwarriors    0.165149
cut            0.146215
sanjaya        0.145163
sini           0.144992
yuyudhana      0.133321
bhurisravas    0.123660
keen           0.118353
grandson       0.116777
shot           0.115106
satwata        0.108736
host           0.107483
rushed         0.104546
carwarrior     0.101800
showers        0.099959
partha         0.097485
kritavarman    0.096659
Name: (7, 6), dtype: float64

In [17]:
comp_strs.unstack()

top_terms  \
pole                                                  0   
pc                                                        
PC0   shafts carwarriors pierced sanjaya army cut dr...   
PC1   rakshasa jarasandha monarchs virata sisupala r...   
PC2   rakshasa deities shafts bhrigu rama vasishtha ...   
PC3   rakshasa sanjaya vidura gandhari bhrigu hope s...   
PC4   jarasandha rakshasa gandhari sanjaya magadha d...   
PC5   sanjaya island hope islands chariot yaksha ser...   
PC6   jarasandha magadha rakshasa sisupala monarchs ...   
PC7   virata matsyas viratas jarasandha uttara matsy...   
PC8   rama andhakas kesava vrishnis sanjaya vrishni ...   
PC9   sanjaya rakshasa virata rama vasishtha maiden ...   

                                                         top_doc           
pole                                                  1        0        1  
pc                                                                         
PC0   deities vidura penances vyasa brahman becomes ...   (7, 6)  (13, 1)  
PC1   deities righteousness becomes penances brahman...   (3, 2)  (12, 3)  
PC2   sanjaya vidura gandhari dussasana hope dhritar...   (1, 9)   (5, 2)  
PC3   jarasandha sisupala monarchs vrishni partha an...   (3, 2)  (1, 18)  
PC4   virata dice dussasana play spake thereupon mat...   (2, 3)   (3, 6)  
PC5   vidura rakshasa drupada town vrishnis monarchs...   (1, 1)  (1, 14)  
PC6   gandhari ladies andhakas vrishnis rama virata ...   (2, 3)  (16, 1)  
PC7   vidura sanjaya maya shafts mansion phalguna ba...   (4, 2)   (1, 8)  
PC8   jarasandha gandhari vidura household ladies pe...  (1, 18)   (2, 4)  
PC9   savitri markandeya vidura misfortune attaineth...   (6, 1)  (3, 14)

In [18]:
COMPS.to_csv(fr"{output_dir}\PCA_Results\{data_prefix}-PCA10_Components.csv",sep = "|")